# YOLO26 vs YOLO11: ASL Detection Benchmark

[![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code)
[![GitHub](https://img.shields.io/badge/GitHub-raimbekovm/yolo26--asl-blue)](https://github.com/raimbekovm/yolo26-asl)

This notebook benchmarks **YOLO26** vs **YOLO11** on American Sign Language (ASL) letter detection.

## What We'll Do
1. Download ASL dataset (26 letter classes)
2. Train YOLO26n on ASL detection
3. Train YOLO11n for comparison
4. Benchmark speed and accuracy
5. Export results

**Author:** Murat Raimbekov  
**License:** Apache 2.0

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q ultralytics>=8.3.0 roboflow

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import torch

from ultralytics import YOLO
import ultralytics

print(f"Ultralytics: {ultralytics.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Download Dataset

We use the **American Sign Language Letters** dataset from Roboflow Universe.
- 26 classes (A-Z)
- ~700 images with bounding box annotations

In [ ]:
from roboflow import Roboflow

# Initialize Roboflow
# Get your API key from https://roboflow.com/settings/api
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")  # Replace with your key

# Download dataset
project = rf.workspace("david-lee-d0rhs").project("american-sign-language-letters")
dataset = project.version(6).download("yolov8")

print(f"Dataset downloaded to: {dataset.location}")

In [ ]:
# Check dataset structure
import yaml

data_yaml = f"{dataset.location}/data.yaml"
with open(data_yaml) as f:
    data_config = yaml.safe_load(f)

print(f"Classes: {data_config['nc']}")
print(f"Names: {data_config['names']}")

# Count images
train_imgs = len(list(Path(f"{dataset.location}/train/images").glob("*.jpg")))
val_imgs = len(list(Path(f"{dataset.location}/valid/images").glob("*.jpg")))
test_imgs = len(list(Path(f"{dataset.location}/test/images").glob("*.jpg")))

print(f"\nTrain: {train_imgs} images")
print(f"Valid: {val_imgs} images")
print(f"Test: {test_imgs} images")

## 3. Train YOLO26

In [ ]:
# Load YOLO26n model
yolo26 = YOLO("yolo26n.pt")

print(f"YOLO26n parameters: {sum(p.numel() for p in yolo26.model.parameters()):,}")

In [ ]:
# Train YOLO26 on ASL dataset
results_26 = yolo26.train(
    data=data_yaml,
    epochs=100,
    imgsz=640,
    batch=16,
    name="yolo26n_asl",
    patience=20,
    save=True,
    plots=True,
    device=0,  # GPU
    workers=4,
    # Augmentation
    augment=True,
    mixup=0.1,
    mosaic=1.0,
)

In [ ]:
# Validate YOLO26
yolo26_best = YOLO("runs/detect/yolo26n_asl/weights/best.pt")
metrics_26 = yolo26_best.val(data=data_yaml)

print(f"\nYOLO26 Results:")
print(f"  mAP50: {metrics_26.box.map50:.4f}")
print(f"  mAP50-95: {metrics_26.box.map:.4f}")

## 4. Train YOLO11 (Baseline)

In [ ]:
# Load YOLO11n model
yolo11 = YOLO("yolo11n.pt")

print(f"YOLO11n parameters: {sum(p.numel() for p in yolo11.model.parameters()):,}")

In [ ]:
# Train YOLO11 on ASL dataset (same settings)
results_11 = yolo11.train(
    data=data_yaml,
    epochs=100,
    imgsz=640,
    batch=16,
    name="yolo11n_asl",
    patience=20,
    save=True,
    plots=True,
    device=0,
    workers=4,
    augment=True,
    mixup=0.1,
    mosaic=1.0,
)

In [ ]:
# Validate YOLO11
yolo11_best = YOLO("runs/detect/yolo11n_asl/weights/best.pt")
metrics_11 = yolo11_best.val(data=data_yaml)

print(f"\nYOLO11 Results:")
print(f"  mAP50: {metrics_11.box.map50:.4f}")
print(f"  mAP50-95: {metrics_11.box.map:.4f}")

## 5. Benchmark Speed

In [ ]:
def benchmark_inference(model, num_runs=100, imgsz=640, device='cuda'):
    """
    Benchmark model inference speed.
    
    Returns:
        dict: Timing results in milliseconds
    """
    # Create dummy input
    dummy = np.random.randint(0, 255, (imgsz, imgsz, 3), dtype=np.uint8)
    
    # Warmup
    for _ in range(10):
        model(dummy, verbose=False, device=device)
    
    # Benchmark
    times = []
    for _ in range(num_runs):
        start = time.perf_counter()
        model(dummy, verbose=False, device=device)
        times.append((time.perf_counter() - start) * 1000)
    
    times = np.array(times)
    return {
        'mean_ms': np.mean(times),
        'std_ms': np.std(times),
        'min_ms': np.min(times),
        'max_ms': np.max(times),
        'fps': 1000 / np.mean(times)
    }

In [ ]:
# Benchmark on GPU
print("Benchmarking on GPU...\n")

bench_26_gpu = benchmark_inference(yolo26_best, num_runs=100, device='cuda')
bench_11_gpu = benchmark_inference(yolo11_best, num_runs=100, device='cuda')

print(f"YOLO26 (GPU): {bench_26_gpu['mean_ms']:.2f} ± {bench_26_gpu['std_ms']:.2f} ms ({bench_26_gpu['fps']:.1f} FPS)")
print(f"YOLO11 (GPU): {bench_11_gpu['mean_ms']:.2f} ± {bench_11_gpu['std_ms']:.2f} ms ({bench_11_gpu['fps']:.1f} FPS)")

gpu_speedup = bench_11_gpu['mean_ms'] / bench_26_gpu['mean_ms']
print(f"\nGPU Speedup: {(gpu_speedup-1)*100:.1f}% faster")

In [ ]:
# Benchmark on CPU
print("Benchmarking on CPU...\n")

bench_26_cpu = benchmark_inference(yolo26_best, num_runs=50, device='cpu')
bench_11_cpu = benchmark_inference(yolo11_best, num_runs=50, device='cpu')

print(f"YOLO26 (CPU): {bench_26_cpu['mean_ms']:.2f} ± {bench_26_cpu['std_ms']:.2f} ms ({bench_26_cpu['fps']:.1f} FPS)")
print(f"YOLO11 (CPU): {bench_11_cpu['mean_ms']:.2f} ± {bench_11_cpu['std_ms']:.2f} ms ({bench_11_cpu['fps']:.1f} FPS)")

cpu_speedup = bench_11_cpu['mean_ms'] / bench_26_cpu['mean_ms']
print(f"\nCPU Speedup: {(cpu_speedup-1)*100:.1f}% faster")

## 6. Results Comparison

In [ ]:
# Create comparison table
results_df = pd.DataFrame({
    'Model': ['YOLO26n', 'YOLO11n'],
    'mAP50': [metrics_26.box.map50, metrics_11.box.map50],
    'mAP50-95': [metrics_26.box.map, metrics_11.box.map],
    'GPU (ms)': [bench_26_gpu['mean_ms'], bench_11_gpu['mean_ms']],
    'GPU (FPS)': [bench_26_gpu['fps'], bench_11_gpu['fps']],
    'CPU (ms)': [bench_26_cpu['mean_ms'], bench_11_cpu['mean_ms']],
    'CPU (FPS)': [bench_26_cpu['fps'], bench_11_cpu['fps']],
})

print("\n" + "="*60)
print("YOLO26 vs YOLO11 - ASL Detection Benchmark")
print("="*60)
print(results_df.to_string(index=False))
print("="*60)

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Accuracy comparison
models = ['YOLO26n', 'YOLO11n']
map50 = [metrics_26.box.map50, metrics_11.box.map50]
map5095 = [metrics_26.box.map, metrics_11.box.map]

x = np.arange(len(models))
width = 0.35

axes[0].bar(x - width/2, map50, width, label='mAP50', color='#2ecc71')
axes[0].bar(x + width/2, map5095, width, label='mAP50-95', color='#3498db')
axes[0].set_ylabel('mAP')
axes[0].set_title('Accuracy Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].legend()
axes[0].set_ylim(0, 1)

# GPU Speed
gpu_times = [bench_26_gpu['mean_ms'], bench_11_gpu['mean_ms']]
colors = ['#e74c3c' if t > min(gpu_times) else '#2ecc71' for t in gpu_times]
axes[1].bar(models, gpu_times, color=colors)
axes[1].set_ylabel('Inference Time (ms)')
axes[1].set_title('GPU Speed (lower is better)')
for i, v in enumerate(gpu_times):
    axes[1].text(i, v + 0.5, f'{v:.1f}ms', ha='center')

# CPU Speed
cpu_times = [bench_26_cpu['mean_ms'], bench_11_cpu['mean_ms']]
colors = ['#e74c3c' if t > min(cpu_times) else '#2ecc71' for t in cpu_times]
axes[2].bar(models, cpu_times, color=colors)
axes[2].set_ylabel('Inference Time (ms)')
axes[2].set_title('CPU Speed (lower is better)')
for i, v in enumerate(cpu_times):
    axes[2].text(i, v + 1, f'{v:.1f}ms', ha='center')

plt.tight_layout()
plt.savefig('benchmark_results.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Sample Predictions

In [ ]:
# Run inference on test images
test_images = list(Path(f"{dataset.location}/test/images").glob("*.jpg"))[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(test_images):
    # Predict with YOLO26
    results = yolo26_best(str(img_path), verbose=False)
    
    # Plot
    annotated = results[0].plot()
    axes[idx].imshow(annotated[..., ::-1])  # BGR to RGB
    axes[idx].axis('off')
    
    # Get prediction
    if len(results[0].boxes) > 0:
        cls_id = int(results[0].boxes.cls[0])
        conf = float(results[0].boxes.conf[0])
        cls_name = results[0].names[cls_id]
        axes[idx].set_title(f"Pred: {cls_name} ({conf:.0%})")

plt.suptitle('YOLO26 ASL Detection Results', fontsize=14)
plt.tight_layout()
plt.savefig('sample_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Export Models

In [ ]:
# Export to ONNX for deployment
yolo26_best.export(format="onnx", imgsz=640, simplify=True)
print("YOLO26 exported to ONNX")

# Save benchmark results
results_df.to_csv('benchmark_results.csv', index=False)
print("Results saved to benchmark_results.csv")

In [ ]:
# Save trained model for HuggingFace
import shutil

output_dir = Path("asl_model")
output_dir.mkdir(exist_ok=True)

shutil.copy("runs/detect/yolo26n_asl/weights/best.pt", output_dir / "yolo26n_asl.pt")
shutil.copy("benchmark_results.png", output_dir)
shutil.copy("benchmark_results.csv", output_dir)

print(f"\nModel and results saved to: {output_dir}")
print("Download this folder to deploy on HuggingFace Spaces!")

## Summary

### Results

| Metric | YOLO26n | YOLO11n |
|--------|---------|--------|
| mAP50 | TBD | TBD |
| mAP50-95 | TBD | TBD |
| GPU Speed | TBD ms | TBD ms |
| CPU Speed | TBD ms | TBD ms |

### Key Findings
- YOLO26 NMS-free architecture enables faster inference
- Both models achieve high accuracy on ASL detection
- CPU speedup is particularly significant for edge deployment

### Links
- **GitHub**: [raimbekovm/yolo26-asl](https://github.com/raimbekovm/yolo26-asl)
- **HuggingFace Demo**: Coming soon

---

**Author:** Murat Raimbekov  
**License:** Apache 2.0